# Suduko

Guidance and inspiration were taken from [*Creating Sudoku Solver with Python and Pyomo in Easy Steps*](https://medium.com/@dhanalakotamohan314/creating-sudoku-solver-with-python-and-pyomo-in-easy-steps-fe22ec916090) by Dhanalakota Mohan.

In [1]:
import re

import numpy as np
import pandas as pd
from pyomo import environ as pyo
from pyomo.dataportal import DataPortal
from pyomo.opt import SolverFactory

## Problem

Find values for the empty cells of a Suduko puzzle.

### Sets

No sets.

### Parameters

No parameters.

### Variables

- $X_{ijk}$: 3D binary matrix set to 1 if the value $k$ should be at row $i$ and column $j$

### Objective

No objective function to minimize or maximize.

### Constraints

- rows: $\sum_i^N x_{ijk} \quad \forall j \in N, \forall k \in N$
- cols: $\sum_j^N x_{ijk} \quad \forall i \in N, \forall k \in N$
- blocks: $\sum_{j=3p-2}^{3p} \sum_{i=3q-2}^{3q} x_{ijk}=1 \quad \forall k \in N, \forall p,q \in [1,3]$ 
- known cells: $x_{ijk}=1 \quad \forall (i,j,k) \in G \hat{=} \{\text{all known cells}\}$

## Data

In [2]:
# The grid indices and possible values.
N = np.arange(1, 9 + 1)

# Known data.
known_cells = pd.DataFrame(
    [
        (1, 7, 2),
        (2, 2, 8),
        (2, 6, 7),
        (2, 8, 9),
        (3, 1, 6),
        (3, 3, 2),
        (3, 7, 5),
        (4, 2, 7),
        (4, 5, 6),
        (5, 4, 9),
        (5, 6, 1),
        (6, 5, 2),
        (6, 8, 4),
        (7, 3, 5),
        (7, 7, 6),
        (7, 9, 3),
        (8, 2, 9),
        (8, 4, 4),
        (8, 8, 7),
        (9, 3, 6),
    ],
    columns=["i", "j", "k"],
)

## Coding the model

In [3]:
model = pyo.ConcreteModel()

### Variable

In [4]:
model.X = pyo.Var(N, N, N, within=pyo.Binary)

### Constraints

In [5]:
model.row_constraint = pyo.ConstraintList()
model.col_constraint = pyo.ConstraintList()
model.block_constraint = pyo.ConstraintList()
model.allcells_constraint = pyo.ConstraintList()
model.knowncells_constraint = pyo.ConstraintList()

# Rows
for i in N:
    for k in N:
        model.row_constraint.add(sum(model.X[i, j, k] for j in N) == 1)

# Columns
for j in N:
    for k in N:
        model.col_constraint.add(sum(model.X[i, j, k] for i in N) == 1)

# Blocks
for i in np.arange(1, 9, 3):
    for j in np.arange(1, 9, 3):
        for k in N:
            model.block_constraint.add(
                sum(
                    model.X[p, q, k]
                    for p in np.arange(i, i + 3)
                    for q in np.arange(j, j + 3)
                )
                == 1
            )

# All cells
for i in N:
    for j in N:
        model.allcells_constraint.add(sum(model.X[i, j, k] for k in N) == 1)

# Known cells
for i, j, k in zip(known_cells["i"], known_cells["j"], known_cells["k"]):
    #     if k != 0:
    model.knowncells_constraint.add(model.X[i, j, k] == 1)

### Objective

In [6]:
model.objective = pyo.Objective(expr=1)

### Solving

In [7]:
opt = SolverFactory("glpk")
solution = opt.solve(model)

    solver failure.


In [8]:
res = np.zeros((9, 9))

for v in model.component_data_objects(pyo.Var, active=True):
    val = v.value
    if val > 0:
        i, j, k = [int(x) for x in re.findall("[0-9]", v.name)]
        res[i - 1, j - 1] = k

print(res)

[[9. 5. 7. 6. 1. 3. 2. 8. 4.]
 [4. 8. 3. 2. 5. 7. 1. 9. 6.]
 [6. 1. 2. 8. 4. 9. 5. 3. 7.]
 [1. 7. 8. 3. 6. 4. 9. 5. 2.]
 [5. 2. 4. 9. 7. 1. 3. 6. 8.]
 [3. 6. 9. 5. 2. 8. 7. 4. 1.]
 [8. 4. 5. 7. 9. 2. 6. 1. 3.]
 [2. 9. 1. 4. 3. 6. 8. 7. 5.]
 [7. 3. 6. 1. 8. 5. 4. 2. 9.]]


---

In [9]:
%load_ext watermark
%watermark -d -u -v -iv -b -h -m

Last updated: 2020-12-21

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 20.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Hostname: JHCookMac.local

Git branch: master

pandas: 1.1.5
re    : 2.2.1
numpy : 1.19.4
pyomo : 5.7.2

